In [20]:
import pandas as pd
import py_stringmatching as sm
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import style
from sklearn.cross_validation import train_test_split
import re
import operator
style.use('ggplot')

# read in csv as dataframe
# c = pd.read_csv("../blocking/candidate_set.csv", encoding="ISO-8859-1", index_col='_id')
c = pd.read_csv("../blocking/candidate_set_CME_matchTest.csv", encoding="ISO-8859-1", index_col='_id')

c.head()

,ltable_id,rtable_id,ltable_Unnamed,ltable_name,ltable_genus,ltable_family,ltable_ecology,ltable_countries,ltable_threat_paragraph,ltable_conservation_paragraph,...,scientific_name_scientific_name_nmw,scientific_name_scientific_name_sw,block_block_lev_dist,block_block_lev_sim,block_block_jar,block_block_jwn,block_block_exm,block_block_jac_qgm_3_qgm_3,rtable_block.1,country_count_sim
_id,,,,,,,,,,,,,,,,,,,,,
25215,1470,3,1470,"adelaide pigmy bluetongue skink, pygmy bluetongue",Tiliqua,Scincidae,NaN,Australia;,NaN,NaN,...,20,20,0,1,1,1,1,1,yes,1
75598,4363,9,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetl...,"[""\n It is listed on Appendix III of the Be...",...,22,22,0,1,1,1,1,1,yes,1
75600,4365,9,4365,NaN,Pelophylax,Ranidae,Terrestrial; Freshwater,China;,\n\n\n The major threat is over-collecting ...,['\n The Shapotou location is within a nati...,...,13,13,0,1,1,1,1,1,yes,1
78778,7543,9,7543,karpathos frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Greece;,\n\n\n The major threat is habitat loss and...,['\n It is listed on Appendix III of the Be...,...,13,13,0,1,1,1,1,1,yes,1
83513,4363,10,4363,albanian water frog,Pelophylax,Ranidae,Terrestrial; Freshwater,Albania; Montenegro;,\n\n\n The major threat is drainage of wetl...,"[""\n It is listed on Appendix III of the Be...",...,22,22,0,1,1,1,1,1,yes,1


In [22]:
matches = 0
nonmatches = 0;

gold = None
DS = None

while matches < 50:
    gold,unlabeled = train_test_split(c, train_size=0.1039, test_size=0.8961)

    #Automatically label the training data
    labels = []
    matches = 0;
    nonmatches = 0;
    
    for index, row in gold.iterrows():
#         print(row)

        if row['rtable_scientific_name'].strip().lower() == row['ltable_scientific_name'].strip().lower():
            labels.append(1)
            matches = matches + 1
        else:
            labels.append(0)
            nonmatches = nonmatches + 1

#     matches = 50
gold['label'] = labels
print('number of matches in training set:', matches)
print('number of nonmatches in training set:', nonmatches)

number of matches in training set: 50
number of nonmatches in training set: 350


/Users/tammi/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
# gold.to_csv('./gold.csv')
gold.to_csv('./gold_CME_test.csv')
# unlabeled.to_csv('./unlabeled.csv')
unlabeled.to_csv('./unlabeled_CME_test.csv')


In [27]:
# remove unnecessary features
# write out gold and unlabeled csvs with only selected features for matcher

# cols_to_keep = ['country_count_sim', 
#                 'status_match', 
#                 'country_overlap', 
#                 'name_name_jac_dlm_dc0_dlm_dc0', 
#                 'name_name_jac_qgm_3_qgm_3']

# FOR NOW, just use these while we wait for other features
cols_to_keep = ['country_count_sim', 
                'name_name_jac_dlm_dc0_dlm_dc0', 
                'name_name_jac_qgm_3_qgm_3']

gold_reduced = gold[cols_to_keep] 
unlabeled_reduced = unlabeled[cols_to_keep]

In [28]:
# gold_reduced.to_csv('./gold_reduced.csv')
gold_reduced.to_csv('./gold_reduced_CME_test.csv')
# unlabeled_reduced.to_csv('./unlabeled_reduced.csv')
unlabeled_reduced.to_csv('./unlabeled_reduced_CME_test.csv')

In [31]:
#using features for matcher
train,test = train_test_split(gold, train_size=0.7, test_size=0.3)
len(train)

train_reduced = train[cols_to_keep] 
test_reduced = test[cols_to_keep]

In [32]:
train.to_csv('./train_CME_test.csv')
test.to_csv('./test_CME_test.csv')
train_reduced.to_csv('./train_reduced_CME_test.csv')
test_reduced.to_csv('./test_reduced_CME_test.csv')